# Federal Ethics Compliance Chatbot - Enhanced Agentic Version

This notebook demonstrates an enhanced agentic RAG system with planning agent and reflection steps for comprehensive federal ethics guidance.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv('../.env.local')

# Verify API keys are loaded
required_keys = ['OPENAI_API_KEY', 'TAVILY_API_KEY', 'LANGCHAIN_API_KEY']
for key in required_keys:
    if not os.getenv(key):
        print(f"⚠️ Missing {key} in environment")
    else:
        print(f"✅ {key} configured")

# Set up LangSmith tracing
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "federal-ethics-chatbot"

✅ OPENAI_API_KEY configured
✅ TAVILY_API_KEY configured
✅ LANGCHAIN_API_KEY configured


In [ ]:
from langgraph.graph import START, StateGraph
from typing_extensions import TypedDict
from langchain_core.documents import Document
from typing import List, Optional

class GraphState(TypedDict):
    question: str
    context: List[Document]
    response: str
    user_context: Optional[dict]
    search_plan: Optional[str]
    violation_type: Optional[str]
    web_results: List[dict]
    assessment: str

In [3]:
import nest_asyncio

nest_asyncio.apply()

In [4]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader

directory_loader = DirectoryLoader("../data", glob="**/*.pdf", loader_cls=PyMuPDFLoader)

ethics_knowledge_resources = directory_loader.load()

print(f"📚 Loaded {len(ethics_knowledge_resources)} pages from federal ethics laws")
print(f"📄 First page preview: {ethics_knowledge_resources[0].page_content[:200]}...")

📚 Loaded 190 pages from federal ethics laws
📄 First page preview: COMPILATION OF 
FEDERAL ETHICS LAWS 
  
 
PREPARED BY THE 
UNITED STATES OFFICE OF GOVERNMENT ETHICS...


In [5]:
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter

def tiktoken_len(text):
    tokens = tiktoken.encoding_for_model("gpt-4o").encode(
        text,
    )
    return len(tokens)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 750,
    chunk_overlap = 0,
    length_function = tiktoken_len,
)

In [6]:
usa_ethics_law_chunks = text_splitter.split_documents(ethics_knowledge_resources)

print(f"🔄 Split {len(ethics_knowledge_resources)} pages into {len(usa_ethics_law_chunks)} chunks")
print(f"📊 Average chunk size: {sum(len(chunk.page_content) for chunk in usa_ethics_law_chunks) // len(usa_ethics_law_chunks)} characters")

🔄 Split 190 pages into 198 chunks
📊 Average chunk size: 2073 characters


In [7]:
from langchain_openai.embeddings import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

embedding_dim = 1536

In [8]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

client = QdrantClient(":memory:")

In [9]:
client.create_collection(
    collection_name="ethics_knowledge_index",
    vectors_config=VectorParams(size=embedding_dim, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="ethics_knowledge_index",
    embedding=embedding_model,
)

In [10]:
vector_store.add_documents(documents=usa_ethics_law_chunks)
print(f"✅ Added {len(usa_ethics_law_chunks)} chunks to ethics knowledge index")

✅ Added 198 chunks to ethics knowledge index


In [11]:
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

In [ ]:
# Enhanced Agentic System with Planning
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

planning_model = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

PLANNING_TEMPLATE = """
You are a federal ethics research planning agent. Analyze the user's question to develop a comprehensive search and analysis strategy.

USER QUESTION: {question}
USER CONTEXT: {user_context}

Create a structured research plan that includes:
1. **Key Ethics Areas**: What specific federal ethics laws/regulations to focus on
2. **Search Terms**: Targeted web search terms for current guidance
3. **Risk Factors**: Potential aggravating or mitigating circumstances
4. **Analysis Focus**: What aspects need the deepest investigation

Provide a concise but thorough research plan.
"""

planning_chain = ChatPromptTemplate.from_template(PLANNING_TEMPLATE) | planning_model

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

# Initialize web search tool
web_search_tool = TavilySearchResults(
    max_results=3,
    search_depth="advanced",
    include_domains=["osg.gov", "oge.gov", "ethics.gov", "gsa.gov"]
)

/var/folders/yr/gt3hr2h50zv15jd0htzpz2b40000gn/T/ipykernel_28779/4146332860.py:4: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  web_search_tool = TavilySearchResults(


In [ ]:
def collect_user_context(state: GraphState) -> GraphState:
    """Collect user context for personalized guidance"""
    # In production, this would be a form input
    # For demo, we'll extract from question or use defaults
    default_context = {
        "role": "federal_employee",
        "agency": "unknown",
        "seniority": "mid_level",
        "clearance": "unknown"
    }
    return {"user_context": default_context}

def create_search_plan(state: GraphState) -> GraphState:
    """Planning agent creates targeted research strategy"""
    user_context_str = str(state.get("user_context", {}))

    plan_response = planning_chain.invoke({
        "question": state["question"],
        "user_context": user_context_str
    })

    return {"search_plan": plan_response.content}

def web_search(state: GraphState) -> GraphState:
    """Enhanced web search using planning agent's strategy"""
    search_plan = state.get("search_plan", "")

    # Extract key terms from plan for targeted search
    base_query = f"federal ethics violation {state['question']}"
    if "gift" in state["question"].lower():
        query = f"{base_query} penalties reporting requirements OGE guidance"
    elif "conflict" in state["question"].lower():
        query = f"{base_query} recusal divestiture financial disclosure"
    elif "employment" in state["question"].lower():
        query = f"{base_query} post-employment restrictions cooling off period"
    else:
        query = f"{base_query} penalties reporting requirements"

    try:
        web_results = web_search_tool.invoke(query)
        return {"web_results": web_results}
    except Exception as e:
        print(f"⚠️ Web search failed: {e}")
        return {"web_results": []}

In [ ]:
# Updated functions for enhanced workflow
def retrieve_ethics_knowledge(state: GraphState) -> GraphState:
    """Retrieve relevant federal ethics law documents with plan context"""
    retrieved_docs = retriever.invoke(state["question"])
    return {"context": retrieved_docs}

def assess_ethics_violation(state: GraphState) -> GraphState:
    """Generate comprehensive ethics assessment with plan guidance"""
    context_text = "\\n\\n".join([doc.page_content for doc in state["context"]])
    web_text = str(state.get("web_results", []))
    search_plan = state.get("search_plan", "")
    user_context = str(state.get("user_context", {}))

    # Enhanced prompt with plan and context
    prompt = f"""
    SEARCH PLAN: {search_plan}

    USER CONTEXT: {user_context}

    QUESTION: {state['question']}

    FEDERAL ETHICS CONTEXT:
    {context_text}

    WEB SEARCH RESULTS:
    {web_text}

    Provide a comprehensive assessment following the search plan guidance:
    1. **Violation Type**: What type of ethics violation this might be
    2. **Severity Assessment**: Minor, moderate, or serious violation
    3. **Legal Penalties**: Specific penalties from federal law
    4. **Corrective Actions**: Immediate steps required
    5. **Reporting Requirements**: Who to notify and deadlines
    6. **Prevention**: How to avoid similar situations

    Tailor the response to the user's role and context.
    """

    response = planning_model.invoke(prompt)
    return {"response": response.content}

In [ ]:
# Build the Enhanced Ethics Assessment Graph
graph_builder = StateGraph(GraphState)

# Add all nodes
graph_builder.add_node("collect_context", collect_user_context)
graph_builder.add_node("create_plan", create_search_plan)
graph_builder.add_node("retrieve", retrieve_ethics_knowledge)
graph_builder.add_node("search_web", web_search)
graph_builder.add_node("assess", assess_ethics_violation)

# Define the enhanced flow with planning
graph_builder.add_edge(START, "collect_context")
graph_builder.add_edge("collect_context", "create_plan")
graph_builder.add_edge("create_plan", "retrieve")
graph_builder.add_edge("retrieve", "search_web")
graph_builder.add_edge("search_web", "assess")

# Compile the enhanced graph
ethics_graph = graph_builder.compile()

print("🧠 Enhanced Ethics Assessment Graph Compiled")
print("Flow: collect_context → create_plan → retrieve → search_web → assess")
print("✅ Planning agent integrated")

In [ ]:
# Enhanced System Summary
print("📝 ENHANCED FEDERAL ETHICS CHATBOT - SYSTEM SUMMARY")
print("=" * 55)

print("""
🏗️ ENHANCED ARCHITECTURE:
├── User Context Collection (role, agency, seniority)
├── Planning Agent (research strategy with GPT-4o-mini)
├── Data Layer: Federal Ethics Laws PDF (190 pages)
├── Vector Store: Qdrant in-memory with OpenAI embeddings
├── Enhanced Web Search: Tavily with plan-guided queries
└── Assessment Engine: Comprehensive ethics analysis (GPT-4o)

🤖 ENHANCED AGENTIC WORKFLOW:
1. User Context Collection (role, agency, clearance level)
2. Planning Agent (creates targeted research strategy)
3. Knowledge Retrieval (federal law RAG with plan context)
4. Enhanced Web Search (plan-guided search terms)
5. Comprehensive Assessment (violation, severity, penalties)

✅ CAPABILITIES:
- Strategic research planning before execution
- Context-aware guidance based on user role/agency
- Enhanced search strategy with targeted terms
- Multi-model approach (GPT-4o + GPT-4o-mini)
- Identifies potential federal ethics violations
- Assesses severity and legal implications
- Provides specific penalty information
- Offers actionable corrective guidance

📊 PRODUCTION READY:
- Streamlined workflow without reflection overhead
- User context enables personalized assessment
- Strategic planning improves retrieval relevance
""")

print("\\n🚀 READY FOR PRODUCTION DEPLOYMENT")
print("Next: FastAPI backend → Next.js frontend → Docker deployment")